In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e1/sample_submission.csv
/kaggle/input/playground-series-s5e1/train.csv
/kaggle/input/playground-series-s5e1/test.csv


In [2]:
train =pd.read_csv("/kaggle/input/playground-series-s5e1/train.csv")

In [3]:
test =pd.read_csv("/kaggle/input/playground-series-s5e1/test.csv")

In [8]:
train.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,date,country,store,product,num_sold
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
1,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0
2,2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0
3,3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0
4,4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0


In [9]:
!pip install scikit-learn

In [11]:
train.columns

Index(['id', 'date', 'country', 'store', 'product', 'num_sold'], dtype='object')

In [12]:
train.count()

id          230130
date        230130
country     230130
store       230130
product     230130
num_sold    221259
dtype: int64

In [13]:
train.isna().sum()

id             0
date           0
country        0
store          0
product        0
num_sold    8871
dtype: int64

In [15]:
train["country"].unique()

array(['Canada', 'Finland', 'Italy', 'Kenya', 'Norway', 'Singapore'],
      dtype=object)

In [41]:
for i in train.columns:
    print(i," : ",train[i].unique())

id  :  [     1      2      3 ... 230127 230128 230129]
country  :  [0 1 2 3 4 5]
store  :  [0 1 2]
product  :  [1 2 3 4 0]
num_sold  :  [ 973.  906.  423. ... 3446. 2266. 3996.]


In [17]:
unique_values = train["country"].unique()
country_dict = {value: index for index, value in enumerate(unique_values)}
country_dict

{'Canada': 0,
 'Finland': 1,
 'Italy': 2,
 'Kenya': 3,
 'Norway': 4,
 'Singapore': 5}

In [20]:
# df[col + '_encoded'] = df[col].map(label_dict)
train["country"] = train["country"].map(country_dict)

In [21]:
train["country"].unique()

array([0, 1, 2, 3, 4, 5])

In [22]:
train["store"].unique()

array(['Discount Stickers', 'Stickers for Less', 'Premium Sticker Mart'],
      dtype=object)

In [23]:
store_values = train["store"].unique()
store_dict = {value: index for index, value in enumerate(store_values)}
train["store"] = train["store"].map(store_dict)

In [24]:
train["store"].unique()

array([0, 1, 2])

In [29]:
product_values = train["product"].unique()
product_dict = {value: index for index, value in enumerate(product_values)}
train["product"] = train["product"].map(product_dict)

In [31]:
train["product"].unique()

array([0, 1, 2, 3, 4])

In [39]:
train.dropna(inplace=True)

In [36]:
train.drop("date",axis=1,inplace=True)

In [42]:
train.drop("id",axis=1,inplace=True)

In [43]:
train.isna().sum()

country     0
store       0
product     0
num_sold    0
dtype: int64

In [44]:
X = train.drop('num_sold', axis=1)  # Features
y = train['num_sold']  # Target variable

In [49]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [47]:
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)

LinearRegression()

In [48]:
coefficients = model_lr.coef_
intercept = model_lr.intercept_

# Print the coefficients and the intercept
print("Coefficients (Weights):", coefficients)
print("Intercept:", intercept)

Coefficients (Weights): [ 54.23127008 273.96017793   9.82803304]
Intercept: 314.6286400346882


In [50]:
y_pred = model_lr.predict(X_val)

# Evaluate the model
mse = mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print("Mean Squared Error:", mse)
print("R^2 Score:", r2)

Mean Squared Error: 419723.09622822085
R^2 Score: 0.12183976708646649


In [55]:
test["country"] = test["country"].map(country_dict)
test["store"] = test["store"].map(store_dict)
test["product"] = test["product"].map(product_dict)

In [57]:
ans=model_lr.predict(test.drop(["id","date"], axis=1))

In [58]:
submission = pd.DataFrame({"id": test.id, "num_sold": ans})
submission

,id,num_sold
0,230130,314.628640
1,230131,324.456673
2,230132,334.284706
3,230133,344.112739
4,230134,353.940772
...,...,...
98545,328675,1133.705346
98546,328676,1143.533379
98547,328677,1153.361412
98548,328678,1163.189445


In [59]:
submission.to_csv('submission.csv', index = False)

In [61]:
from sklearn.ensemble import RandomForestRegressor

model_rf=RandomForestRegressor(n_estimators=100, random_state=42)

In [62]:
model_rf.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [64]:
ans_rf=model_rf.predict(test.drop(["id","date"], axis=1))

In [65]:
submission_rf = pd.DataFrame({"id": test.id, "num_sold": ans_rf})
submission_rf.to_csv('submission_rf.csv', index = False)